In [ ]:
import pandas as pd
import nltk
import string
import re
from gensim.models.fasttext import FastText
from time import time
import gensim
%matplotlib inline

In [ ]:
tokenizer = nltk.data.load('tokenizers/punkt/russian.pickle')
df = pd.read_csv("wsd_datasets/texts.csv")

In [ ]:
text = df["text"].str.replace(r'([^\w\s]+)', ' \\1 ').str.strip()

In [ ]:
text = text.astype(str)
text = text.str.lower()
text = " ".join(map(str, (review for review in text)))
print ("There are {} words in the combination of all review.".format(len(text)))

In [ ]:
def tokenize(text):
    russian_stopwords = set(nltk.corpus.stopwords.words('russian'))

    punctuation = re.compile(r"[" + string.punctuation + string.ascii_letters + string.digits + "]")

    words = nltk.word_tokenize(text)

    words = [word for word in words if word not in russian_stopwords]

    return [word for
            word in words
            if not re.search(punctuation, word)]

In [ ]:
text_tokens = tokenize(text)

In [ ]:
text = nltk.Text(text_tokens)
print(type(text))

In [ ]:
text_raw = " ".join(text)
with open('wsd_datasets/text.txt', 'w') as textfile:
    textfile.write(text_raw)
    textfile.close()

In [ ]:
data = gensim.models.word2vec.LineSentence('wsd_datasets/text.txt')

In [ ]:
%%time

ft_model = gensim.models.KeyedVectors.load("models/fasttext-1.4gb/model.model")
t = time()
ft_model.build_vocab(data, progress_per=10000, update=True)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))
ft_model.train(data, total_examples=ft_model.corpus_count, epochs=5, report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

In [ ]:
%%time
ft_model = FastText(data, vector_size=100, window=5, min_count=5, workers=4,sg=1)

In [ ]:
ft_model.save("models/fasttext.model")

In [ ]:
ft_model.wv.most_similar(positive=["депозит"], topn=20)

In [ ]:
ft_model["санация"]